In [1]:
!nbstripout --keep-output llama2.ipynb

In [ ]:
from huggingface_hub import login
login("")

In [ ]:
!pip install trl

In [ ]:

!pip install -U bitsandbytes>=0.46.1

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer,SFTConfig

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"

In [ ]:
dataset = load_dataset(dataset_name, split="train")

In [ ]:
compute_dtype = getattr(torch, 'float16')

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,device_map="auto",offload_folder="offload",offload_state_dict=True)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

# def tokenize_function(example):
#     return tokenizer(
#         example["text"],
#         truncation=True,
#         max_length=512,
#         padding="max_length",
#     )

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
sft_config = SFTConfig(
   !zip -r finetuned_model.zip /kaggle/working/results
    num_train_epochs=1,

    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,

    optim="paged_adamw_8bit",

    logging_steps=25,
    save_steps=0,
    report_to="tensorboard",

    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,

    fp16=False,
    bf16=True,

    dataset_text_field="text",
    packing=False,
)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=sft_config,
)

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

2026-02-28 09:56:55.795871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772272615.966863     243 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772272616.017449     243 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772272616.419453     243 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772272616.419486     243 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772272616.419490     243 computation_placer.cc:177] computation placer alr

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': None}.


Step,Training Loss
25,1.590806
50,1.324404
75,1.272339
100,1.329144
125,1.216773
150,1.212227
175,1.249519
200,1.272353
225,1.210593
250,1.186626


TrainOutput(global_step=250, training_loss=1.2864782562255859, metrics={'train_runtime': 9211.0112, 'train_samples_per_second': 0.109, 'train_steps_per_second': 0.027, 'total_flos': 1.675794781200384e+16, 'train_loss': 1.2864782562255859})

In [ ]:
model.save_pretrained("llama2-guanaco-finetuned")
tokenizer.save_pretrained("llama2-guanaco-finetuned")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('llama2-guanaco-finetuned/tokenizer_config.json',
 'llama2-guanaco-finetuned/chat_template.jinja',
 'llama2-guanaco-finetuned/tokenizer.json')

inference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_enable_fp32_cpu_offload=True
)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/llama2_finetune/results/checkpoint-250"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Problem solved: verifying model works but shows repetition

In [ ]:
prompt = "<s>[INST] Explain transformers simply. [/INST] "
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST] Explain transformers simply. [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain transformers simply [/INST]  [INST] Explain


Problem solved: prompt echoing + infinite repetition

In [ ]:
prompt = "<s>[INST] Explain transformers simply. [/INST] "
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id
)
response = tokenizer.decode(
    outputs[0][inputs.input_ids.shape[-1]:],
    skip_special_tokens=True
)

print(response)



[OUT] 1: What is a Transformer? [/OUT]  20-35%
	[/IN] [/OUT][/EXP] 2: How do I implement Transformers in my codebase? [/OUT] 94+%
	[/IN] [/OUT][/EXP] 3: Why are Transformers important for the future of Python? [/OUT] 86+%
	[/IN] [/OUT][/EXP] [/END]



Problem solved: dataset formatting tokens appearing

In [ ]:
prompt = "<s>[INST] Explain transformers simply. [/INST] "
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

bad_words_ids = tokenizer(
    ["[OUT]", "[EXP]", "[END]", "[IN]"],
    add_special_tokens=False
).input_ids
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
    bad_words_ids=bad_words_ids
)
response = tokenizer.decode(
    outputs[0][inputs.input_ids.shape[-1]:],
    skip_special_tokens=True
)

print(response)



[HELP] Transformer is a class for working with lists and objects in the form of "pairs". 

To start, you must create an object: 

```javascript
var t = new Transform(obj)
t.name = 'T'; // this will be called when obj changes
// ...
```

Transformer can convert from one type to another by calling .toObject() or .fromObject(). You may pass any number of arguments as parameters. This function also takes care of some additional work like converting strings into numbers (if they are numbers). 

```javascript
t.toObject([10,20], 3); // 75  4    -2
t.fromObject(['10', ['20'], 3]);     // [ 89 ,      [] ]

t.toObject('a=1');         // undefined    {a:1
